In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# Nạp dữ liệu

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
try:
    df = pd.read_csv('/content/drive/MyDrive/Project_DA_TIMA/Data/Tima_CRM_Handled_Python.csv')
    print("Tải dữ liệu thành công.")
except FileNotFoundError:
    print("Lỗi: Không tìm thấy file 'Tima_CRM_Handled - Copy.csv'. Vui lòng kiểm tra lại đường dẫn.")
    exit()

Tải dữ liệu thành công.


# Lựa chọn các đặc trưng và biến mục tiêu

In [4]:
features = [
    'TS_CREDIT_SCORE_V2', 'SoTienDKVayBanDau', 'Salary', 'Gender',
    'CustomerAge', 'LongestOverdue', 'HasBadDebt', 'HasLatePayment',
    'ProductCreditName', 'CityName', 'HinhThucCuTru', 'JobName'
]
df['IsBadDebt'] = np.where(df['TrangThai'] == 'Nợ Xấu', 1, 0)
target = 'IsBadDebt'

In [5]:
# Lọc dữ liệu để huấn luyện mô hình (loại bỏ các dòng có giá trị thiếu ở các cột quan trọng)
df_model = df.dropna(subset=['TS_CREDIT_SCORE_V2', 'Salary']).copy()

# Tách dữ liệu

In [6]:
X = df_model[features]
y = df_model[target]


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Kích thước tập huấn luyện: {X_train.shape}")
print(f"Kích thước tập kiểm tra: {X_test.shape}")
print("\n")

Kích thước tập huấn luyện: (1555, 12)
Kích thước tập kiểm tra: (389, 12)




# 3. Phân tích và Mô hình hóa Dự báo

In [8]:
# Xác định các cột số và cột phân loại
numerical_features = X.select_dtypes(include=np.number).columns.tolist()
categorical_features = X.select_dtypes(exclude=np.number).columns.tolist()

# Tạo pipeline xử lý
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)])

# Xây dựng pipeline hoàn chỉnh với mô hình Logistic Regression
model_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                 ('classifier', LogisticRegression(random_state=42, class_weight='balanced', max_iter=1000))])

# Huấn luyện mô hình
model_pipeline.fit(X_train, y_train)

print("Mô hình Hồi quy Logistic đã được huấn luyện thành công.")
print("\n")

Mô hình Hồi quy Logistic đã được huấn luyện thành công.




#  4. Xây dựng Mô hình Tối ưu hóa / Hệ thống Đề xuất

In [9]:
# Dự đoán xác suất nợ xấu trên tập kiểm tra
y_pred_proba = model_pipeline.predict_proba(X_test)[:, 1]

# Xây dựng hàm đề xuất dựa trên ngưỡng rủi ro
def prescriptive_model(probability):
    if probability < 0.4:
        return "Luồng Xanh: Ưu tiên Duyệt nhanh"
    elif 0.4 <= probability < 0.7:
        return "Luồng Vàng: Yêu cầu Thẩm định Chuẩn"
    else:
        return "Luồng Đỏ: Rủi ro cao - Thẩm định Kỹ / Đề xuất phương án thay thế"

# Áp dụng mô hình đề xuất
recommendations = [prescriptive_model(p) for p in y_pred_proba]

# Tạo DataFrame kết quả để phân tích
results_df = X_test.copy()
results_df['Actual_IsBadDebt'] = y_test
results_df['Predicted_Probability'] = y_pred_proba
results_df['Recommendation'] = recommendations

print("Hệ thống đề xuất đã được xây dựng:")
print(results_df[['SoTienDKVayBanDau', 'Salary', 'TS_CREDIT_SCORE_V2', 'Actual_IsBadDebt', 'Predicted_Probability', 'Recommendation']].head())

Hệ thống đề xuất đã được xây dựng:
      SoTienDKVayBanDau    Salary  TS_CREDIT_SCORE_V2  Actual_IsBadDebt  \
1377          2500000.0   6000000               500.0                 1   
964          10000000.0   5000000               592.0                 0   
419          10000000.0  12000000               675.0                 0   
246          10000000.0  10000000               685.0                 0   
1721          5000000.0  10000000               702.0                 0   

      Predicted_Probability                                     Recommendation  
1377               0.840894  Luồng Đỏ: Rủi ro cao - Thẩm định Kỹ / Đề xuất ...  
964                0.243355                    Luồng Xanh: Ưu tiên Duyệt nhanh  
419                0.032409                    Luồng Xanh: Ưu tiên Duyệt nhanh  
246                0.194021                    Luồng Xanh: Ưu tiên Duyệt nhanh  
1721               0.003936                    Luồng Xanh: Ưu tiên Duyệt nhanh  


# 5. Đánh giá và Kiểm tra Mô hình

In [10]:
# 5.1 Đánh giá Mô hình Dự báo
print("--- Đánh giá Mô hình Dự báo ---")
y_pred = model_pipeline.predict(X_test)
print("Báo cáo phân loại:\n", classification_report(y_test, y_pred))
print(f"Điểm ROC AUC: {roc_auc_score(y_test, y_pred_proba):.4f}\n")

# 5.2 Đánh giá Mô hình Đề xuất (Mô phỏng tác động kinh doanh)
print("--- Đánh giá Mô hình Đề xuất (Mô phỏng) ---")

# Giả định các tham số tài chính
INTEREST_RATE = 0.20  # Lợi nhuận 20% trên khoản vay thành công
LOSS_RATE = 1.00      # Mất 100% gốc nếu vỡ nợ
COST_OF_REVIEW = 50000 # Chi phí thẩm định kỹ cho 1 hồ sơ (VND)

def simulate_profit(df_results):
    total_profit = 0
    for index, row in df_results.iterrows():
        loan_amount = row['SoTienDKVayBanDau']
        is_bad_debt = row['Actual_IsBadDebt']
        recommendation = row['Recommendation']

        # Quyết định dựa trên đề xuất
        approved = True
        if "Luồng Đỏ" in recommendation:
            approved = False # Giả định Luồng Đỏ bị từ chối/không giải ngân

        if approved:
            if is_bad_debt:
                # Mất tiền gốc
                total_profit -= loan_amount * LOSS_RATE
            else:
                # Thu được lợi nhuận
                total_profit += loan_amount * INTEREST_RATE

            # Thêm chi phí thẩm định cho luồng Vàng
            if "Luồng Vàng" in recommendation:
                total_profit -= COST_OF_REVIEW

    return total_profit

# Kịch bản 1: Không dùng mô hình (Duyệt tất cả)
profit_as_is = simulate_profit(results_df.assign(Recommendation="Duyệt tất cả"))

# Kịch bản 2: Có dùng mô hình đề xuất
profit_to_be = simulate_profit(results_df)

print(f"Lợi nhuận kỳ vọng (Mô phỏng trên tập test) - Kịch bản KHÔNG DÙNG MÔ HÌNH: {profit_as_is:,.0f} VNĐ")
print(f"Lợi nhuận kỳ vọng (Mô phỏng trên tập test) - Kịch bản CÓ DÙNG MÔ HÌNH:    {profit_to_be:,.0f} VNĐ")
print(f"==> Mức tăng lợi nhuận kỳ vọng: {profit_to_be - profit_as_is:,.0f} VNĐ")
print("\n")

--- Đánh giá Mô hình Dự báo ---
Báo cáo phân loại:
               precision    recall  f1-score   support

           0       0.99      0.80      0.88       363
           1       0.24      0.88      0.37        26

    accuracy                           0.80       389
   macro avg       0.61      0.84      0.63       389
weighted avg       0.94      0.80      0.85       389

Điểm ROC AUC: 0.8828

--- Đánh giá Mô hình Đề xuất (Mô phỏng) ---
Lợi nhuận kỳ vọng (Mô phỏng trên tập test) - Kịch bản KHÔNG DÙNG MÔ HÌNH: 517,144,000 VNĐ
Lợi nhuận kỳ vọng (Mô phỏng trên tập test) - Kịch bản CÓ DÙNG MÔ HÌNH:    520,894,000 VNĐ
==> Mức tăng lợi nhuận kỳ vọng: 3,750,000 VNĐ




# 6. Phân tích Kết quả

Dựa trên kết quả mô phỏng, việc áp dụng mô hình phân luồng rủi ro mang lại lợi ích tài chính rõ rệt.  

**Khuyến nghị chính:**  

1. **Triển khai Hệ thống Phân luồng Tự động:**  
   Tích hợp mô hình vào CRM để tự động gán nhãn "Xanh", "Vàng", "Đỏ" cho mọi hồ sơ vay mới.  
   - **Luồng Xanh** (<40% rủi ro): Đẩy nhanh quy trình phê duyệt.  
   - **Luồng Vàng** (40%-70% rủi ro): Thẩm định theo quy trình chuẩn.  
   - **Luồng Đỏ** (>70% rủi ro): Yêu cầu thẩm định cấp cao, tư vấn khách hàng giảm số tiền vay hoặc chuyển đổi sản phẩm.  

2. **Điều chỉnh Chính sách Sản phẩm:**  
   - Đối với các sản phẩm có rủi ro cao như *Cầm cố xe máy KCC*, *Vay theo sim*, cần áp dụng biểu phí rủi ro riêng hoặc tăng yêu cầu về điểm tín dụng đầu vào.  

3. **Tối ưu hóa Marketing:**  
   - Tập trung ngân sách quảng cáo vào tệp khách hàng có đặc điểm tương tự nhóm "Luồng Xanh" (có điểm tín dụng tốt, thu nhập ổn định).  

4. **Thu thập thêm dữ liệu:**  
   - Cần bổ sung trường dữ liệu *Lãi suất* (`InterestRate`) cho mỗi khoản vay. Dữ liệu này sẽ cho phép xây dựng mô hình tối ưu hóa lãi suất, giúp tối đa hóa lợi nhuận trên từng phân khúc rủi ro.
